In [1]:
!pip install -r requirements.txt

# این نوتبوک نوتبوک اصلی است و دیتای مورد نیاز آن حدود ۸ مگ است و برای ران کردن همین فایل های موجود در همین فایل های ارسالی کافی است.
## ران کردن این نوتبوک ملاحظه خاصی ندارد. 

In [2]:
import codecs
import seaborn as sb
import tqdm
import json
from __future__ import unicode_literals
from hazm import *
from nltk import FreqDist
import pandas as pd
import nltk
from matplotlib import pyplot as plt
import numpy as np
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_analysis(token, mode = 0):
    if mode == 0:
        return lemmatizer.lemmatize(token)
    if mode == 1 and mode == 10:
        return stemmer.stem(token)
    if mode == 2:
        return normalizer.normalize(token)
    if mode == 3:
        return sent_tokenize(token)
    if mode == 4:
        return word_tokenize(token)
def flatten(txt,mode = False , f = None):
    s = []
    if not mode:
        for sen in txt:
            for x in sen:
                w = x.strip()
                s.append(w)
    else:
        for sen in txt:
            for x in sen:
                w = x.strip()
                if f(w):
                    s.append(w)
    return s
def merge_stop_words(stopwL):
    res = []
    for pwl in stopwL:
        for wl in pwl:
            if wl.strip() not in res:
                res.append(wl.strip())
    return res

stopw = codecs.open('./module_data/stopwords.txt','r','utf-8').readlines()
my_own_stopw = codecs.open('./module_data/my_stopwords.txt','r','utf-8').readlines()

stopw = merge_stop_words([stopw,my_own_stopw])
stopwords = [get_analysis(x.strip()) for x in stopw ]
stopwords_dict = {x:True for x in stopwords}
source_dir = lambda name: f"./data/{name}/{name}.txt"
tagger = POSTagger(model='./module_data/resources/postagger.model')

In [3]:
poets = ["hafez","saadi","moulavi","ferdousi","khayyam","nezami"]
txt_data = {}
for i in poets:
    txt =  open(source_dir(i),"r",encoding="utf-8").readlines()
    txt_data[i] = [s.strip() for s in txt]



In [4]:
normalized = {}
print("Normalizer")
for key in txt_data:
    normalized[key] = [get_analysis(txt.strip(),2)  for txt in tqdm.tqdm(txt_data[key])]
sentence_tokenized = {}
print("sentence_tokenized")
for key in normalized:
    sentence_tokenized[key] = [get_analysis(sen,3) for sen in tqdm.tqdm(normalized[key])]
    sentence_tokenized[key] = flatten(sentence_tokenized[key])
print("word_tokenized")
word_tokenized = {}
for key in sentence_tokenized:
    word_tokenized[key] = [get_analysis(sen,4) for sen in tqdm.tqdm(sentence_tokenized[key])]

Normalizer


100%|██████████| 55555/55555 [00:03<00:00, 16229.36it/s]


sentence_tokenized


100%|██████████| 55555/55555 [00:00<00:00, 155493.35it/s]


word_tokenized


100%|██████████| 55595/55595 [00:00<00:00, 58590.69it/s]


In [5]:
lemmatized_words = {}
print("lemmatized_words")
for key in word_tokenized:
    lemmatized_words[key] = [[get_analysis(w.strip()) for w in sen if tagger.tag([w.strip()])[0][1] not in ["V","P"]] for sen in tqdm.tqdm(word_tokenized[key])]
    lemmatized_words[key] = flatten(lemmatized_words[key])

lemmatized_words


100%|██████████| 55595/55595 [00:06<00:00, 8118.43it/s]


In [6]:
f = lambda x : not stopwords_dict.get(x,False)
words = {}
for key in lemmatized_words:
    words[key] = [w.strip() for w in tqdm.tqdm(lemmatized_words[key])  if f(w.strip())]

100%|██████████| 242754/242754 [00:00<00:00, 1643061.86it/s]


In [7]:
dataframe = {}
freq_most = 150
for key in lemmatized_words:
    dataframe[key] = FreqDist(words[key]).most_common(freq_most)      
freq_analysis = pd.DataFrame(dataframe)

In [8]:
freq_analysis.iloc[0:40]

,hafez,saadi,moulavi,ferdousi,khayyam,nezami
0,"(حافظ, 509)","(دست, 1312)","(جان, 7718)","(شاه, 4617)","(خاک, 69)","(شاه, 1197)"
1,"(جان, 306)","(دوست, 884)","(عشق, 3719)","(بد, 2220)","(جهان, 66)","(دست, 1136)"
2,"(باد, 284)","(جان, 686)","(آب, 2632)","(سپاه, 2124)","(دست, 65)","(کار, 1042)"
3,"(چشم, 258)","(سعدی, 669)","(چشم, 2347)","(جهان, 2097)","(خوش, 58)","(جهان, 1023)"
4,"(عشق, 254)","(چشم, 663)","(دست, 2213)","(لشکر, 1805)","(ملک, 53)","(آب, 942)"
5,"(دست, 242)","(عشق, 591)","(حق, 2124)","(سخن, 1782)","(نیکو, 50)","(سخن, 771)"
6,"(خوش, 237)","(شب, 500)","(صد, 2096)","(کار, 1594)","(مرد, 50)","(جان, 771)"
7,"(غم, 218)","(سخن, 466)","(جهان, 2062)","(ایران, 1577)","(ماه, 50)","(خاک, 756)"
8,"(یار, 210)","(دشمن, 441)","(خوش, 1936)","(مرد, 1519)","(عمر, 49)","(باد, 732)"
9,"(زلف, 208)","(غم, 433)","(شب, 1704)","(تخت, 1480)","(زمین, 49)","(شب, 702)"


In [9]:
def vasf_ext(sentence):
    grammar = r"""
      NVASF: {<N|Ne><AJ>}
      NEZAF: {<N|Ne><N|Ne>}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))

In [10]:
tarkibat = {}
for key in word_tokenized:
    tarkibat[key] = {}
    NVASF = []
    NEZAF = []
    for sentence in tqdm.tqdm(word_tokenized[key]):
        tree=vasf_ext(tagger.tag(sentence))
        for subtree in tree.subtrees():
            if subtree.label() == 'NVASF':
                NVASF.append(subtree.leaves())
            if subtree.label() == 'NEZAF':
                NEZAF.append(subtree.leaves())   
    tarkibat[key]["NVASF"]  = [' '.join([x[0],y[0]]) for x,y in NVASF]
    tarkibat[key]["NEZAF"] =[' '.join([x[0],y[0]]) for x,y in NEZAF]  

100%|██████████| 55595/55595 [00:15<00:00, 3493.94it/s]


In [11]:
dataframe_NEZAF = {}
dataframe_NVASF = {}
freq_most = 50
for key in tarkibat:
    dataframe_NEZAF[key] = FreqDist(tarkibat[key]["NEZAF"]).most_common(freq_most)    
    dataframe_NVASF[key] = FreqDist(tarkibat[key]["NVASF"]).most_common(freq_most)    
freq_analysis_NEZAF = pd.DataFrame(dataframe_NEZAF)
freq_analysis_NVASF = pd.DataFrame(dataframe_NVASF)

In [12]:
freq_analysis_NVASF


,hafez,saadi,moulavi,ferdousi,khayyam,nezami
0,"(چشم بد, 10)","(سرو روان, 19)","(چشم بد, 63)","(شب تیره, 173)","(ملوک عجم, 11)","(چشم بد, 44)"
1,"(دل خوش, 7)","(یار عزیز, 13)","(آب روان, 34)","(دل پر, 146)","(ایزد تعالی, 6)","(چرخ بلند, 20)"
2,"(ماه نو, 6)","(خدای عزوجل, 11)","(شیر نر, 29)","(گرز گران, 76)","(روی نیکو, 6)","(گل سرخ, 17)"
3,"(شاه شجاع, 6)","(خدای عز, 11)","(حق تعالی, 26)","(یزدان پاک, 60)","(دیدار نیکو, 4)","(تنی چند, 17)"
4,"(تازه نو, 6)","(سرو بلند, 11)","(جهان پر, 23)","(باد سرد, 58)","(حالی خوش, 4)","(سپهر بلند, 15)"
5,"(لب جوی, 5)","(روز روشن, 9)","(رطل گران, 21)","(چرخ بلند, 57)","(سبزه نشین, 3)","(چرخ کبود, 15)"
6,"(مرغ زیرک, 5)","(عمر عزیز, 9)","(لاله زار, 19)","(وزان جایگه, 52)","(خدای تعالی, 3)","(ماه نو, 15)"
7,"(عمر دراز, 4)","(یار مهربان, 9)","(رو ترش, 19)","(خورشید تابان, 50)","(خط نیکو, 3)","(چشم روشن, 14)"
8,"(وادی ایمن, 4)","(جواب تلخ, 9)","(خوی بد, 18)","(دلی پر, 47)","(ایزد تبارک, 3)","(کبک دری, 14)"
9,"(سرو خرامان, 4)","(سرو خرامان, 9)","(گل سرخ, 18)","(پیل مست, 46)","(روبه آرام, 2)","(دل تنگ, 13)"


In [13]:
freq_analysis_NEZAF

,hafez,saadi,moulavi,ferdousi,khayyam,nezami
0,"(باد صبا, 18)","(غم دل, 19)","(شمس تبریزی, 163)","(شاه جهان, 121)","(دفع مضرتش, 12)","(سر تخت, 26)"
1,"(یاد باد, 18)","(حدیث عشق, 17)","(شمس تبریز, 146)","(ازان پس, 105)","(زبان حال, 6)","(آب حیوان, 26)"
2,"(سر زلف, 16)","(خاک پای, 16)","(شمس دین, 130)","(نزدیک شاه, 100)","(چرخ فلک, 5)","(شاه جهان, 22)"
3,"(خون دل, 14)","(باد صبا, 14)","(آب حیات, 112)","(جهان آفرین, 90)","(سر سبزه, 4)","(آب حیات, 20)"
4,"(دل حافظ, 12)","(سر کوی, 14)","(جان جان, 106)","(شاه ایران, 84)","(عبدالله طاهر, 4)","(سر خاک, 19)"
5,"(پیر مغان, 12)","(دل ریش, 14)","(شمس الدین, 88)","(ایران سپاه, 81)","(رفتن دی, 4)","(زمین بوس, 16)"
6,"(سر کوی, 10)","(دوست دست, 14)","(شمس الحق, 86)","(دل شاه, 79)","(زیر زمین, 4)","(خاره سنگ, 16)"
7,"(کام دل, 9)","(سر زلف, 13)","(آب حیوان, 62)","(تخت عاج, 77)","(بزبان پهلوی, 4)","(سهی سرو, 15)"
8,"(خون جگر, 9)","(دل سعدی, 13)","(تبریز شمس, 50)","(گردش روزگار, 72)","(سال پادشاهی, 4)","(چاره ساز, 14)"
9,"(خاک ره, 8)","(آب چشم, 13)","(شمس حق, 49)","(خاقان چین, 71)","(کارگه کوزه‌گری, 4)","(کام دل, 14)"


In [14]:
def pi_plot(data,label,title):
    plt.figure(figsize =(15, 12))
    plt.title(title)
    plt.pie(data, labels = label,autopct='%1.1f%%',shadow = True,)
    plt.savefig(f'poets_charts/{title}.png')
    plt.clf()

types = ["_words","_NEZAF","_NVASF"]
NP_type = {k:{} for k in types }
for t in types:
    d = pd.read_csv(f"type_dataset/unique_most{t}.csv")[["word","type"]]
    for i,row in d.iterrows():
        NP_type[t][row["word"]] = row["type"]

def create_plot(key,data,t):
    cols = {"philosophy":0,"love":0,"daily":0,"religion":0,"others":0}
    for w1,w2 in data:
        w = NP_type[t].get(w1,"others")
        cols[w] = cols[w]+int(w2)
    pi_plot([cols[key] for key in cols],[key for key in cols],f"{key}{t}")
    pi_plot([cols[key] for key in cols if key != "others" ],[key for key in cols if key != "others"],f"{key}{t}_no_others")


for key in dataframe:
    create_plot(key,dataframe[key],"_words")
    create_plot(key,dataframe_NEZAF[key],"_NEZAF")
    create_plot(key,dataframe_NVASF[key],"_NVASF")


/tmp/ipykernel_46175/975698000.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize =(15, 12))


<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

...

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>

<Figure size 1080x864 with 0 Axes>